<center>
<p style = "color : red; font-size : 35px; font-family : 'Comic Sans MS'; ">
    <strong>        
        Linear Regression on Logarithmic Data and Polynomial Features (Visualize)
    </strong>
</p>
</center>

In [1]:
import pandas as pd
import numpy as np
import plotly.graph_objects as go
import plotly.express as px
import scipy.stats as stats

from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import root_mean_squared_error, r2_score
from sklearn.preprocessing import PolynomialFeatures

In [53]:
from sklearn.metrics._scorer import _SCORERS

print(_SCORERS.keys())

dict_keys(['explained_variance', 'r2', 'max_error', 'matthews_corrcoef', 'neg_median_absolute_error', 'neg_mean_absolute_error', 'neg_mean_absolute_percentage_error', 'neg_mean_squared_error', 'neg_mean_squared_log_error', 'neg_root_mean_squared_error', 'neg_root_mean_squared_log_error', 'neg_mean_poisson_deviance', 'neg_mean_gamma_deviance', 'd2_absolute_error_score', 'accuracy', 'top_k_accuracy', 'roc_auc', 'roc_auc_ovr', 'roc_auc_ovo', 'roc_auc_ovr_weighted', 'roc_auc_ovo_weighted', 'balanced_accuracy', 'average_precision', 'neg_log_loss', 'neg_brier_score', 'positive_likelihood_ratio', 'neg_negative_likelihood_ratio', 'adjusted_rand_score', 'rand_score', 'homogeneity_score', 'completeness_score', 'v_measure_score', 'mutual_info_score', 'adjusted_mutual_info_score', 'normalized_mutual_info_score', 'fowlkes_mallows_score', 'precision', 'precision_macro', 'precision_micro', 'precision_samples', 'precision_weighted', 'recall', 'recall_macro', 'recall_micro', 'recall_samples', 'recall_w

In [ ]:
np.random.seed(42)
X = np.linspace(1, 1000, 1000).reshape(-1, 1)

noise_level = 0.2

y = np.log(X) + np.random.normal(0, noise_level, size=len(X)).reshape(-1, 1)
# - 0: Mean of the distribution (center of the bell curve)
# - noise_level: Standard deviation (spread of the distribution)
# - size=len(x): Number of random numbers to generate, matching the length of `X`

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42, test_size=.2)

In [13]:
fig = go.Figure()

fig.add_trace(go.Scatter(x=X_train.flatten(), y=y_train.flatten(), mode='markers', name='Train data'))

fig.add_trace(go.Scatter(x=X_test.flatten(), y=y_test.flatten(), mode='markers', name='test data'))

fig.update_layout(height=700)

fig.show()

In [14]:
model1 = LinearRegression()
model1.fit(X_train, y_train)

y_pred = model1.predict(X_test)

print(f'RMSE : {root_mean_squared_error(y_test, y_pred):2f}')
print(f'r2 : {r2_score(y_test, y_pred):2f}')


RMSE : 0.429498
r2 : 0.770829


# is this model good?

<strong>One effective way to determine if a linear model performs well is by analyzing the residual error.</strong>

What is residual error, and how does it help us?


By analyzing the residuals, you can assess how well your model captures the data:

- Low Residual Error indicates that the model accurately fits the data.
- Random Pattern of Residuals suggests the model correctly captures relationships within the data.
- <i style="color: red; font-size:24px; text-decoration: underline;">Patterns or Trends in Residuals may highlight missing variables, model limitations, or data issues, suggesting the need for adjustments either in the model or in the data itself.<i>

In [15]:
residual_error = y_test - y_pred

fig = go.Figure()

fig.add_trace(go.Scatter(x=X_test.flatten(), y=residual_error.flatten(), mode='markers', name='residual_error'))

fig.add_shape(
    type="line",
    x0=min(X_test)[0], y0=0,
    x1=max(X_test)[0], y1=0,
    line=dict(color="Red", width=2, dash='dashdot'),
    name='y=0'
)

fig.update_layout(
    title="Scatter Plot for residual_error",
    xaxis_title="X",
    yaxis_title="Y",
    height=700
)

fig.show()


## Can you spot the pattern? 🧐🧐
Let’s try a different approach: the `normal probability plot` (`QQ-plot`).

In [16]:
qq_x, qq_y = stats.probplot(residual_error.flatten(), dist="norm")[0]

fig = go.Figure()

fig.add_trace(go.Scatter(
    x=qq_x,
    y=qq_y,
    mode='markers',
    marker=dict(size=5),
    name='Sample Quantiles'
))

fig.add_shape(
    type="line",
    x0=-3, y0=-3,
    x1=3, y1=3,
    line=dict(color="Red", width=2, dash='dashdot'),
    name='Normal Reference Line'
)


fig.update_layout(
    title="Normal Probability Plot of Residuals error (QQ-plot)",
    xaxis_title="Theoretical Quantiles",
    yaxis_title="Sample Quantiles",
    template="plotly_white",
    showlegend=False,
    height=700
)

fig.show()

<p style="font-family:verdana; font-size:24px">
In this QQ Plot, you can see that the residuals from our simple linear regression model don’t quite match a normal distribution. Ideally, if the errors were normally distributed, the points would fall along the red line. Here, though, they deviate from it.
</p>

<p style="font-family:verdana; font-size:24px">
This suggests that our simple model didn’t capture all the patterns in the data. To improve, we might consider more complex models like polynomial regression or machine learning methods that could better handle these patterns.
</p>

## Now, what should we do? 😭😭

We have 2 options:

- **Option 1**: Use a complex model.
- **Option 2**: Enhance our features by creating new ones, such as polynomial features. By adding polynomial features, we can capture more complex patterns without switching to a more complicated model. This can improve our model's performance by allowing it to fit non-linear relationships in the data.

Let's go with Option 2 and try feature engineering to see if it helps! 🤞🤞

Since we only have one input feature, `x`, the question is, what degree should we choose for **PolynomialFeatures**? 🤔

Let’s keep it simple and use a loop to test degrees from 2 to 13 and see which one works best. This way, we can explore how adding different powers of `x` might improve the model without using complex models. 😎

In [33]:

df = pd.DataFrame(columns=['Degree', 'RMSE', 'r2'])

for degree in range(2, 14):
    poly = PolynomialFeatures(degree=degree)
    poly.fit(X_train)

    X_train_poly = poly.transform(X_train)
    X_test_poly = poly.transform(X_test)

    model2 = LinearRegression()
    model2.fit(X_train_poly, y_train)

    y_pred = model2.predict(X_test_poly)

    rmse = root_mean_squared_error(y_test, y_pred)
    r2 = r2_score(y_test, y_pred)
    
    df.loc[len(df.index)] = (degree, rmse, r2)

df.sort_values(by=['r2'], ascending=False)

,Degree,RMSE,r2
3,5.0,0.221048,0.939297
2,4.0,0.226618,0.936199
1,3.0,0.254626,0.919455
4,6.0,0.271369,0.908513
0,2.0,0.293021,0.893332
5,7.0,0.300978,0.887460
6,8.0,0.351935,0.846127
7,9.0,0.415771,0.785244
8,10.0,0.469763,0.725846
9,11.0,0.509830,0.677085


<p style="font-family=verdana">
Although degree 5 gives the best results, it’s often safer to step back by one degree to avoid overfitting. Choosing degree 4 helps create a more general model that captures the main patterns without becoming too specific to our current data. This way, the model is more likely to perform well on new data too!
</p>


In [31]:
fig = px.line(df, x="Degree", y=["RMSE", "r2"], markers=True, labels={
    "value": "Metrics",
    "Degree": "Polynomial Degree"
}, title="Polynomial Degree vs RMSE and R2")
fig.update_traces(mode="lines+markers")
fig.show()

<div class="alert alert-block alert-info" style="font-family:verdana; line-height: 1.7em;">
📌 &nbsp; The data you're seeing here is test data, which allows us to try out different model settings. But in many real cases, we don’t have the luxury of testing all options on test data. Usually, we only have training and validation data, and due to large volumes, it’s not always practical to test every possible configuration. Since this is an educational notebook, though, we're making an exception and exploring all the possibilities to give you a clear view of how different setups perform!
</div>



# create the best model 🤩🤩

In [34]:
poly = PolynomialFeatures(degree=4)
poly.fit(X_train)

X_train_poly = poly.transform(X_train)
X_test_poly = poly.transform(X_test)

model2 = LinearRegression()
model2.fit(X_train_poly, y_train)

y_pred = model2.predict(X_test_poly)

rmse = root_mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)
print(f'RMSE : {rmse:.2f}')
print(f'r2 : {r2:.2f}')

RMSE : 0.23
r2 : 0.94


In [41]:
pd.DataFrame(list(zip(poly.get_feature_names_out(), model2.coef_[0])), columns=['Feature', 'Coefficient'])

,Feature,Coefficient
0,1,0.000000e+00
1,x0,2.521433e-02
2,x0^2,-6.768186e-05
3,x0^3,8.117584e-08
4,x0^4,-3.445460e-11


<p style = "color : #f55c47; font-size : 35px; font-family : 'Comic Sans MS';">
    <strong>
        If you like this, Please do upvote.😊🌹
    </strong>
</p>